## **GUÍA MODELADO DOCPLEX**

La siguiente es un guía para entender la interfaz de docplex y como trabajar con los 5 items estándar de los modelos de optimización.

1. Conjuntos
2. Parámetros
3. Variables
4. Función Objetivo
5. Restricciones

También se muestran algunas funcionalidades que ofrece docplex para configuración y posprocesamiento

##### **Conjuntos**


In [ ]:
## Listas 😊
A = ["a1", "a2", "a3"]

## Conjuntos ❓
B = {1, 2, 3, 4, 5, 6, 7}


##### **Parámetros**


In [ ]:
## Diccionario 1
parametro_1 = {
    (1, 1, 1): 123,
    (1, 1, 2): 178,
    (1, 2, 1): 165,
    (1, 2, 2): 143,
    (2, 1, 1): 154,
    (2, 1, 2): 151,
    (2, 2, 1): 163,
    (2, 2, 2): 132,
}

## Diccionario 2
parametro_2 = {1: {1: 2133, 2: 1344}, 2: {1: 1455, 2: 2200}}

## Dataframe ❓
import pandas

data = {
    ("Categoria A", "Producto 1"): [10, 5],
    ("Categoria A", "Producto 2"): [15, 3],
    ("Categoria B", "Producto 1"): [20, 7],
    ("Categoria B", "Producto 2"): [25, 2],
}

df = pandas.DataFrame.from_dict(data, orient="index", columns=["Precio", "Cantidad"])
df.index = pandas.MultiIndex.from_tuples(df.index)
df


In [ ]:
parametro_1[2, 2, 1]

In [ ]:
parametro_2[2][1]

In [ ]:
df.loc[("Categoria A", "Producto 1"), "Precio"]


##### **Creación de modelo**


In [ ]:
## Importanto DoCplex
from docplex.mp.model import Model

In [ ]:
# Inicializando el modelo
model = Model("el nombre de mi modelo")


##### **Variables**


##### Variables simples

In [ ]:
model.continuous_var(-5, 100, "la variable continua")

In [ ]:
model.binary_var("la variable binaria")


In [ ]:
model.integer_var(7, 100, "la varible entera")

##### Variables dict

In [ ]:
var_a = model.continuous_var_dict(A)
var_a

In [ ]:
var_a["a1"]

##### Variables como lista

In [ ]:
model.continuous_var_list(A)

##### Variables múltiples conjuntos

In [ ]:
var_ab = model.binary_var_dict([(a, b) for a in A for b in B])
var_ab

In [ ]:
var_ab["a3", 5]

In [ ]:
model.binary_var_matrix(A, B)

In [ ]:
model.binary_var_cube(A, A, B)

##### **Función Objetivo**


In [ ]:
x = model.continuous_var()
y = model.continuous_var()

## FO explícita
model.maximize(x + y)

## FO compacta
model.minimize(model.sum(var_a[a] for a in A))

##### **Restricciones**


In [ ]:
## Restricción explicita
model.add_constraint(x + y <= 100, ctname="ct_1")

In [ ]:
## Restricción compacta
for a in A:
    model.add_constraint(var_a[a] <= 100, ctname="el nomobre de la restriccion")

In [ ]:
for a in A:
    for b in B:
        model.add_constraint(var_ab[a, b] <= 1)

In [ ]:
model.add_constraints([var_a[a] <= 100 for a in A])

##### **POSPROCESAMIENTO Y CONFIGURACIONES**

##### Resolver un modelo

In [ ]:
from docplex.mp.model import Model

mdl = Model(name="buses")

nbbus40 = mdl.integer_var(name="nbBus40")
nbbus30 = mdl.integer_var(name="nbBus30")
mdl.minimize(nbbus40 * 500 + nbbus30 * 400)
mdl.add_constraint(nbbus40 * 40 + nbbus30 * 30 >= 300, "kids")

solution = mdl.solve()

##### Resolver un modelo mostrando logs

In [ ]:
from docplex.mp.model import Model

mdl = Model(name="buses")

nbbus40 = mdl.integer_var(name="nbBus40")
nbbus30 = mdl.integer_var(name="nbBus30")
mdl.add_constraint(nbbus40 * 40 + nbbus30 * 30 >= 300, "kids")
mdl.minimize(nbbus40 * 500 + nbbus30 * 400)


mdl.solve(log_output=True)


##### Objeto solution

In [ ]:
from docplex.mp.model import Model

mdl = Model(name="buses")

nbbus40 = mdl.integer_var(name="nbBus40")
nbbus30 = mdl.integer_var(name="nbBus30")
mdl.add_constraint(nbbus40 * 40 + nbbus30 * 30 >= 300, "kids")
mdl.minimize(nbbus40 * 500 + nbbus30 * 400)


solution = mdl.solve()

solution


In [ ]:
solution.get_objective_value()

In [ ]:
solution.get_value(nbbus40)

In [ ]:
solution.display()


In [ ]:
solution.solve_details


In [ ]:
solution.solve_status


In [ ]:
solution.export_as_json_string()


##### Exportar .lp

In [ ]:
## Exportar a LP
from docplex.mp.model import Model

mdl = Model(name="buses")


nbbus40 = mdl.integer_var(name="nbBus40")
nbbus30 = mdl.integer_var(name="nbBus30")
mdl.add_constraint(nbbus40 * 40 + nbbus30 * 30 >= 300, "kids")
mdl.minimize(nbbus40 * 500 + nbbus30 * 400)

mdl.export("./buses.lp")


##### Limitar tiempo de ejecución

In [ ]:
## Limete de tiempo
from docplex.mp.model import Model

mdl = Model(name="buses")


nbbus40 = mdl.integer_var(name="nbBus40")
nbbus30 = mdl.integer_var(name="nbBus30")
mdl.add_constraint(nbbus40 * 40 + nbbus30 * 30 >= 300, "kids")
mdl.minimize(nbbus40 * 500 + nbbus30 * 400)

mdl.set_time_limit(20)  # The same
mdl.solve()

print("time limit = ", mdl.get_time_limit())  # The same

for v in mdl.iter_integer_vars():
    print(v, " = ", v.solution_value)


##### Limitar número de mejoras de solucion

In [ ]:
## N-ésima solución encontrada
mdl = Model(name="buses")

nbbus40 = mdl.integer_var(name="nbBus40")
nbbus30 = mdl.integer_var(name="nbBus30")
mdl.add_constraint(nbbus40 * 40 + nbbus30 * 30 >= 300, "kids")
mdl.minimize(nbbus40 * 500 + nbbus30 * 400)

mdl.parameters.mip.limits.solutions = 1
mdl.solve()

print("int sol limit = ", mdl.parameters.mip.limits.solutions.get())


for v in mdl.iter_integer_vars():
    print(v, " = ", v.solution_value)
